In [231]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import math as m
from scipy.stats import unitary_group

In [303]:
class Model(object):
    def __init__(self):
        self.UF  = tf.Variable([[1,8],[-1,1]],dtype = tf.float32,trainable = True)  # ext. micromotion operator
        self.UF  = normalisation(self.UF)
 
def normalisation(U_):
    # U_  (in) original matrix
    #     (out)  matrix with normalised vectors
    normaU_ = tf.sqrt(tf.math.reduce_sum(tf.multiply(U_,U_,1),axis=0))
    U__ = tf.math.truediv(U_,normaU_)
    return U__
 

def Gram_Schmidt(U_):
    #U__ = normalisation(U_)
    #U_  = U_.assign(U__)#normalisation(U_)
    #Y = tf.Variable([[1,8],[-8,1]],dtype = tf.float32,trainable = True)  # ext. micromotion operator
    Y = tf.TensorArray(tf.float32, size=tf.shape(U_).numpy()[0]) 
    Y_ = []
    for i in range(0,tf.shape(U_).numpy()[0]):
        
        temp_vec = U_[:,i]
        
        for inY in Y_:
            norminY    = tf.math.reduce_sum(tf.multiply(inY,inY,0)).numpy()
            projec_vec = tf.math.multiply(tf.math.reduce_sum(tf.multiply(U_[:,i],inY,0))/norminY,inY)
            temp_vec   = tf.math.subtract(temp_vec,projec_vec)
              
        temp_vec = normalisation(temp_vec)     
        #print(i,temp_vec)
        Y_.append(temp_vec)    
        #Y = Y.write(i, temp_vec)
        #U_ = U_[:,i].assign(temp_vec)
        #print(Y.stack())
        #Y.assign(temp_vec)    
        #print(Y_)
    #Y = np.transpose(np.stack(Y))
    return Y_#.stack()


def MatrixOperations(U_):
    #c = tf.TensorArray(tf.float32, size=2) 
    trace = tf.reduce_sum(tf.linalg.tensor_diag_part(U_))    
    det   = U_[0,0]*U_[1,1] - U_[0,1]*U_[1,0]
    Y = tf.Variable([[0,0],[0,0]],dtype=tf.float32)
    Y = tf.transpose(U_)
    c = [trace,det]
    #c = c.write(0, trace)
    #c = c.write(1, det)
    #print(c.concat)
    return c#trace,det

def Gram_Schmidt_eltos(U_):
    D = tf.shape(U_).numpy()[0]
    Y = []
    for k in range(0,D):
        for i in range(0,D):

            g = 0
            #print(i,k)
            for j in range(0,k):
            #    print(j)
                for l in range(0,D):
                    g += U_[l,j]*U_[l,k]*U_[i,j]
                   # print(g)
           # print(i,k,g)
            Y.append(U_[i,k]-g)
            
        
    return Y


model = Model()

UF = tf.constant([[-1,3],[3,4]], dtype=tf.float32)
UF = normalisation(UF)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(UF)
    #loss_value = MatrixOperations(UF)  
    loss_value = Gram_Schmidt_eltos(UF)
    print(loss_value)
   # print(loss_value[1])
print(tape.gradient(loss_value[0], [UF]))
print(tape.gradient(loss_value[1], [UF]))
print(tape.gradient(loss_value[2], [UF]))
print(tape.gradient(loss_value[3], [UF]))

print(tape.gradient(loss_value, [UF]))


[<tf.Tensor: id=23209, shape=(), dtype=float32, numpy=-0.31622776>, <tf.Tensor: id=23215, shape=(), dtype=float32, numpy=0.94868326>, <tf.Tensor: id=23251, shape=(), dtype=float32, numpy=0.78>, <tf.Tensor: id=23287, shape=(), dtype=float32, numpy=0.2600001>]
[<tf.Tensor: id=23295, shape=(2, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 0.]], dtype=float32)>]
[<tf.Tensor: id=23300, shape=(2, 2), dtype=float32, numpy=
array([[0., 0.],
       [1., 0.]], dtype=float32)>]
[<tf.Tensor: id=23326, shape=(2, 2), dtype=float32, numpy=
array([[-0.37947327,  0.9       ],
       [ 0.25298223,  0.29999998]], dtype=float32)>]
[<tf.Tensor: id=23352, shape=(2, 2), dtype=float32, numpy=
array([[-0.56921   ,  0.29999998],
       [-1.3281565 ,  0.10000008]], dtype=float32)>]
[<tf.Tensor: id=23410, shape=(2, 2), dtype=float32, numpy=
array([[ 0.05131674,  1.2       ],
       [-0.0751743 ,  0.4000001 ]], dtype=float32)>]
